Code used to validate the behavior of a fuzzyy PD controller for reservoir level control.

The function calculatePower() describes the embedded PLC logic responsible for the control.

The skfuzzyy package was used to compare the centroid method with the weighted average and the mean of maximums.

In [ ]:
def calculatePower(error, deltaError, errorTolerance, deltaErrorTolerance):

    """ Error Membership Function """
    
    # LNE: Large Negative Error, SNE: Small Negative Error, ZEE: Zero Error, 
    # SPE: Small Positive Error, LPE: Large Positive Error
    
    LNE, SNE, ZEE, SPE, LPE = 0, 0, 0, 0, 0

    if (error <= (-2 * errorTolerance)):
        LNE = 1
    elif (error > (-2 * errorTolerance)) and (error <= -errorTolerance):
        LNE = -(error + errorTolerance) / errorTolerance
        SNE = 1 - LNE
    elif (error > -errorTolerance) and (error <= 0):
        ZEE = (error + errorTolerance) / errorTolerance
        SNE = 1 - ZEE
    elif (error > 0) and (error <= errorTolerance):
        SPE = error / errorTolerance
        ZEE = 1 - SPE
    elif (error > errorTolerance) and (error <= (2 * errorTolerance)):
        LPE = (error - errorTolerance) / errorTolerance
        SPE = 1 - LPE
    elif (error > (2 * errorTolerance)):
        LPE = 1

    """ Delta Error Membership Function """
    
    # LNDE: Large Negative Delta Error, SNDE: Small Negative Delta Error, 
    # ZEDE: Zero Delta Error, SPDE: Small Positive Delta Error, LPDE: Large Positive Delta Error
    
    LNDE, SNDE, ZEDE, SPDE, LPDE = 0, 0, 0, 0, 0

    if (deltaError <= (-2 * deltaErrorTolerance)):
        LNDE = 1
    elif (deltaError > (-2 * deltaErrorTolerance)) and (deltaError <= -deltaErrorTolerance):
        SNDE = (deltaError + 2 * deltaErrorTolerance) / deltaErrorTolerance
        LNDE = 1 - SNDE
    elif (deltaError > -deltaErrorTolerance) and (deltaError <= 0):
        ZEDE = (deltaError + deltaErrorTolerance) / deltaErrorTolerance
        SNDE = 1 - ZEDE
    elif (deltaError > 0) and (deltaError <= deltaErrorTolerance):
        SPDE = deltaError / deltaErrorTolerance
        ZEDE = 1 - SPDE
    elif (deltaError > deltaErrorTolerance) and (deltaError <= (2 * deltaErrorTolerance)):
        LPDE = (deltaError - deltaErrorTolerance) / deltaErrorTolerance
        SPDE = 1 - LPDE
    elif (deltaError > (2 * deltaErrorTolerance)):
        LPDE = 1

    """ RULES """
    
    # Rule activation for two defuzzyification methods:
    # Method 1 (F1): Weighted Average of Maximums
    # Method 2 (F2): Weighted Average (Sum of all activations)
    
    VHP1_F, HP1_F, MP1_F, LP1_F, VLP1_F = 0, 0, 0, 0, 0
    VHP2_F, HP2_F, MP2_F, LP2_F, VLP2_F = 0, 0, 0, 0, 0

    """ Line 1 (If Delta Error is Large Negative) """
    if (LNE != 0) and (LNDE != 0):
        VHP1_F = max(VHP1_F, min(LNE, LNDE))
        VHP2_F += min(LNE, LNDE)
    if (SNE != 0) and (LNDE != 0):
        MP1_F = max(MP1_F, min(SNE, LNDE))
        MP2_F += min(SNE, LNDE)
    if (ZEE != 0) and (LNDE != 0):
        LP1_F = max(LP1_F, min(ZEE, LNDE))
        LP2_F += min(ZEE, LNDE)
    if (SPE != 0) and (LNDE != 0):
        LP1_F = max(LP1_F, min(SPE, LNDE))
        LP2_F += min(SPE, LNDE)
    if (LPE != 0) and (LNDE != 0):
        LP1_F = max(LP1_F, min(LPE, LNDE))
        LP2_F += min(LPE, LNDE)

    """ Line 2 (If Delta Error is Small Negative) """
    if (LNE != 0) and (SNDE != 0):
        HP1_F = max(HP1_F, min(LNE, SNDE))
        HP2_F += min(LNE, SNDE)
    if (SNE != 0) and (SNDE != 0):
        MP1_F = max(MP1_F, min(SNE, SNDE))
        MP2_F += min(SNE, SNDE)
    if (ZEE != 0) and (SNDE != 0):
        LP1_F = max(LP1_F, min(ZEE, SNDE))
        LP2_F += min(ZEE, SNDE)
    if (SPE != 0) and (SNDE != 0):
        LP1_F = max(LP1_F, min(SPE, SNDE))
        LP2_F += min(SPE, SNDE)
    if (LPE != 0) and (SNDE != 0):
        VLP1_F = max(VLP1_F, min(LPE, SNDE))
        VLP2_F += min(LPE, SNDE)

    """ Line 3 (If Delta Error is Zero) """
    if (LNE != 0) and (ZEDE != 0):
        VHP1_F = max(VHP1_F, min(LNE, ZEDE))
        VHP2_F += min(LNE, ZEDE)
    if (SNE != 0) and (ZEDE != 0):
        HP1_F = max(HP1_F, min(SNE, ZEDE))
        HP2_F += min(SNE, ZEDE)
    if (ZEE != 0) and (ZEDE != 0):
        MP1_F = max(MP1_F, min(ZEE, ZEDE))
        MP2_F += min(ZEE, ZEDE)
    if (SPE != 0) and (ZEDE != 0):
        LP1_F = max(LP1_F, min(SPE, ZEDE))
        LP2_F += min(SPE, ZEDE)
    if (LPE != 0) and (ZEDE != 0):
        VLP1_F = max(VLP1_F, min(LPE, ZEDE))
        VLP2_F += min(LPE, ZEDE)

    """ Line 4 (If Delta Error is Small Positive) """
    if (LNE != 0) and (SPDE != 0):
        HP1_F = max(HP1_F, min(LNE, SPDE))
        HP2_F += min(LNE, SPDE)
    if (SNE != 0) and (SPDE != 0):
        MP1_F = max(MP1_F, min(SNE, SPDE))
        MP2_F += min(SNE, SPDE)
    if (ZEE != 0) and (SPDE != 0):
        LP1_F = max(LP1_F, min(ZEE, SPDE))
        LP2_F += min(ZEE, SPDE)
    if (SPE != 0) and (SPDE != 0):
        LP1_F = max(LP1_F, min(SPE, SPDE))
        LP2_F += min(SPE, SPDE)
    if (LPE != 0) and (SPDE != 0):
        VLP1_F = max(VLP1_F, min(LPE, SPDE))
        VLP2_F += min(LPE, SPDE)

    """ Line 5 (If Delta Error is Large Positive) """
    if (LNE != 0) and (LPDE != 0):
        HP1_F = max(HP1_F, min(LNE, LPDE))
        HP2_F += min(LNE, LPDE)
    if (SNE != 0) and (LPDE != 0):
        MP1_F = max(MP1_F, min(SNE, LPDE))
        MP2_F += min(SNE, LPDE)
    if (ZEE != 0) and (LPDE != 0):
        LP1_F = max(LP1_F, min(ZEE, LPDE))
        LP2_F += min(ZEE, LPDE)
    if (SPE != 0) and (LPDE != 0):
        LP1_F = max(LP1_F, min(SPE, LPDE))
        LP2_F += min(SPE, LPDE)
    if (LPE != 0) and (LPDE != 0):
        VLP1_F = max(VLP1_F, min(LPE, LPDE))
        VLP2_F += min(LPE, LPDE)

    """ Defuzzyification """
    
    # Weighted Average of Maximums
    
    P1 = ((8.33 * VLP1_F) + (25.0 * LP1_F) + (50.0 * MP1_F) + (75.0 * HP1_F) + (91.66 * VHP1_F)) / \
         (VLP1_F + LP1_F + MP1_F + HP1_F + VHP1_F)
    
    # Weighted Average
    
    P2 = ((8.33 * VLP2_F) + (25.0 * LP2_F) + (50.0 * MP2_F) + (75.0 * HP2_F) + (91.66 * VHP2_F)) / \
         (VLP2_F + LP2_F + MP2_F + HP2_F + VHP2_F)

    return P1, P2

In [ ]:
import skfuzzy as fuzzy
import skfuzzy.control as ctrl
import numpy as np

# --- Configuration ---

error_tolerance = 5.5
e_max, e_min = 200, -200
e_range = np.arange(e_min, e_max, 0.1)

delta_error_tolerance = 0.1
de_max, de_min = 1, -1
de_range = np.arange(de_min, de_max, 0.1)

# --- Antecedents and Consequents ---

error = ctrl.Antecedent(e_range, 'error')
delta_error = ctrl.Antecedent(de_range, 'delta_error')
power = ctrl.Consequent(np.arange(0, 100, 0.1), 'power')

# --- Error Membership Functions ---

error['LNE'] = fuzzy.trapmf(error.universe, [e_min, e_min, -2 * error_tolerance, -error_tolerance])
error['SNE'] = fuzzy.trimf(error.universe, [-2 * error_tolerance, -error_tolerance, 0])
error['ZE']  = fuzzy.trimf(error.universe, [-error_tolerance, 0, error_tolerance])
error['SPE'] = fuzzy.trimf(error.universe, [0, error_tolerance, 2 * error_tolerance])
error['LPE'] = fuzzy.trapmf(error.universe, [error_tolerance, 2 * error_tolerance, e_max, e_max])

# --- Delta Error Membership Functions ---

delta_error['LNDE'] = fuzzy.trapmf(delta_error.universe, [de_min, de_min, -2 * delta_error_tolerance, -delta_error_tolerance])
delta_error['SNDE'] = fuzzy.trimf(delta_error.universe, [-2 * delta_error_tolerance, -delta_error_tolerance, 0])
delta_error['ZDE']  = fuzzy.trimf(delta_error.universe, [-delta_error_tolerance, 0, delta_error_tolerance])
delta_error['SPDE'] = fuzzy.trimf(delta_error.universe, [0, delta_error_tolerance, 2 * delta_error_tolerance])
delta_error['LPDE'] = fuzzy.trapmf(delta_error.universe, [delta_error_tolerance, 2 * delta_error_tolerance, de_max, de_max])

# --- Output Membership Functions (Power) ---

power['VL'] = fuzzy.trimf(power.universe, [0, 0, 25])
power['L']  = fuzzy.trimf(power.universe, [0, 25, 50])
power['M']  = fuzzy.trimf(power.universe, [25, 50, 75])
power['H']  = fuzzy.trimf(power.universe, [50, 75, 100])
power['VH'] = fuzzy.trimf(power.universe, [75, 100, 100])

# --- Rule Base Setup ---

rules_list = []

# Tuning matrix (Mapping inputs to output sets)
tuning = ['VH', 'M', 'L', 'L', 'L',
          'H',  'M', 'L', 'L', 'VL',
          'VH', 'H', 'M', 'L', 'VL',
          'H',  'M', 'L', 'L', 'VL',
          'H',  'M', 'L', 'L', 'VL']

counter = 0
for de_term in delta_error.terms:
    for e_term in error.terms:
        rules_list.append(ctrl.Rule(delta_error[de_term] & error[e_term], power[tuning[counter]]))
        counter += 1

# --- Controller Simulation ---

control_sys = ctrl.ControlSystem(rules_list)
controller = ctrl.ControlSystemSimulation(control_sys)

setpoint = 50.0
previous_pv = 0
levels = np.arange(0, 100, 0.3)

# Data storage for comparison
p_weighted_max, p_weighted_avg, p_skfuzzy = [], [], []

for current_pv in levels:
    err = current_pv - setpoint
    der = current_pv - previous_pv

    # Manual Calculation (PLC Logic)
    pot1, pot2 = calculatePower(err, der, error_tolerance, delta_error_tolerance)
    p_weighted_max.append(pot1)
    p_weighted_avg.append(pot2)

    # Scikit-fuzzy Calculation
    controller.input['error'] = err
    controller.input['delta_error'] = der
    controller.compute()

    p_skfuzzy.append(controller.output['power'])
    previous_pv = current_pv

In [ ]:
import matplotlib.pyplot as plt

# Convert lists to numpy arrays for calculations
p_weighted_max = np.array(p_weighted_max)
p_weighted_avg = np.array(p_weighted_avg)
p_skfuzzy = np.array(p_skfuzzy)

# Calculating differences (Residuals)
diff_weighted_max = p_skfuzzy - p_weighted_max
diff_weighted_avg = p_skfuzzy - p_weighted_avg

# Plotting the Power Comparison
plt.figure(figsize=(10, 6))
plt.plot(levels, p_weighted_max, label='Weighted Average of Maximums (Manual)')
plt.plot(levels, p_weighted_avg, label='Weighted Average (Manual)')
plt.plot(levels, p_skfuzzy, label='Centroid (skfuzzy)', linestyle='--')
plt.plot([setpoint, setpoint], [0, 90], c="r", linestyle='--', label="setpoint")

plt.title('Comparison of Defuzzification Methods for Reservoir Control')
plt.xlabel('Reservoir Level (cm)')
plt.ylabel('Output Power (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(10, 7))
# ax = fig.add_subplot(111, projection='3d')

# # X: levels
# # Y: valor constante
# # Z: p_skfuzzy
# ax.plot(levels, [0.3] * len(levels), zs=p_skfuzzy, zdir='z', label='Fuzzy Power Curve', color='blue')

# ax.set_xlabel('Reservoir Level (cm)')
# ax.set_ylabel('Time/Step Proxy')
# ax.set_zlabel('Output Power (%)')
# ax.set_title('3D Visualization of Fuzzy Control')
# ax.legend()

# plt.show()